In [ ]:
# IPython Config
%config IPCompleter.greedy=True 
# Standard library imports
import os
import time
import ctypes
import configparser

# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
import zhinst.core

# Qcodes imports
import qcodes as qc
from qcodes.dataset import Measurement, do1d, initialise_or_create_database_at, load_or_create_experiment, plot_dataset
from qcodes.station import Station
from qcodes.tests.instrument_mocks import DummyInstrument, DummyInstrumentWithMeasurement

# Picosdk imports
from picosdk.ps5000a import ps5000a as ps
from picosdk.functions import adc2mV, assert_pico_ok
import picoscope_module as pm

# Laboneq and related imports
import laboneq
from laboneq.simple import Simple  # <-- Import specific functionalities if wildcard import is avoided
from laboneq.pulse_sheet_viewer.interactive_psv import interactive_psv

# Other imports
import helpers
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

DO_EMULATION = False # run in emulation mode by default 

# Load config file
config = configparser.ConfigParser()
config.read('config230823.ini')
with open(config['DataPaths']['descriptor_path'], 'r') as file:
    descriptor = file.read()

# Labber related parameters
qc.config.add('stabilitymap', config['DataPaths']['labber_data_path'], value_type='string', description='Name of the stability diagram in labber', default=None)

# Setting up the qcodes configuration
qc.logger.start_all_logging()
gate_instr = DummyInstrument('gate_instr', gates=['gate1', 'gate2'])
data_acq_mock = DummyInstrumentWithMeasurement(name='data_acq_mock', setter_instr=gate_instr)
station = qc.Station(data_acq_mock, gate_instr)

# Setting up a database
db_file_path = os.path.join(config['DataPaths']['data_save_path'],'savingtest.db')
initialise_or_create_database_at(db_file_path)
exp = load_or_create_experiment(
    experiment_name=config['DataPaths']['experiment_name'],
    sample_name=config['DataPaths']['sample_name']
)

# create and connect to HDAWG session
device_setup = DeviceSetup.from_descriptor(descriptor,server_host="127.0.0.1",server_port="8004", setup_name="ZI_HDAWG")
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)
if not session.connection_state.emulated:
    instrument_serial = device_setup.instrument_by_uid("device_hdawg").address
    device = session.devices[instrument_serial]
    device.triggers.out[2].delay(23.9e-9)

#picoscope 
params = {key: float(value) for key, value in config['PicoscopeParameters'].items()}
pico_sampling_rate = (params['timebase'] - 3) / 62500000
preTriggerSamples=int(np.ceil(params['pretriggersamples']))
postTriggerSamples = int(np.ceil(params['posttriggersamples_val'] / pico_sampling_rate))
maxSamples = preTriggerSamples + postTriggerSamples

chandle, status = pm.initialize_picoscope()
status, chARange, chBRange = pm.setup_channels(chandle,status)
status,maxADC = pm.setup_trigger(chandle, chARange, chBRange, params['triggerlevel'],status)

In [ ]:
# 5. Setting up measurements
meas = Measurement(exp=exp)
meas.register_parameter(gate_instr.gate1)
meas.register_parameter(gate_instr.gate2)
meas.register_parameter(data_acq_mock.v1, setpoints=(gate_instr.ch1,gate_instr.ch2), paramtype='array')
meas.write_period = 0.1

#defining parameters 
times_map = {
    'pulse_time': 10e-3, 
    'dead_time': 10e-3, 
    'trigger_time': 1e-3
}

points_map={
    'initial_point': [0.0, 0.0],
    'anchor_point': [-0.0, 0.0],
    'load_point':    [0.0, 0.0],
    'readout_point': [0.0, 0.0], 
    'control_point': [0.0, 0.0],}

qc.config.add('points_map', points_map, value_type='object', description='Mapping of points', default=None)
qc.config.add('times_map', points_map, value_type='object', description='Mapping of times', default=None)

compress_level_pulse=pulse_library.const(uid="compress_level",length=times_map['pulse_time'],amplitude=1,can_compress=True)

# assign amplitude to received variable
NUM_REP = 50

x_p = np.linspace(0.1, -0.1, 2)
y_p = np.linspace(0.1, -0.1, 2)

x_mesh, y_mesh = np.meshgrid(x_p, y_p)
tensor_product = np.column_stack((x_mesh.ravel(), y_mesh.ravel()))

#define pulse shapes
#@pulse_library.register_pulse_functional
#def ramp(x,start=0,stop=1,**_):
#    pulse=start+ (stop-start)*(x+1)/2
#    return pulse

# 6. Running the measurements
with meas.run() as datasaver:
    for received_variable in range(len(x_p)*len(y_p)):
        read_amplitude=tensor_product[received_variable]
        
        # Experiment
        exp = Experiment(
            "Pulse Experiment",
            signals=[
                ExperimentSignal("gate1"),
                ExperimentSignal("gate2"),
            ],
        )

        with exp.acquire_loop_rt(
            uid=("pulse"), count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
        ):
            with exp.section(
                uid=("unload"),
                length=times_map['pulse_time'],
                trigger={"gate1":{"state":1}},
                alignment=SectionAlignment.LEFT,
            ):
                exp.play(signal="gate1",pulse=compress_level_pulse,amplitude=-0.1)
                exp.play(signal="gate2",pulse=compress_level_pulse,amplitude=-0.1)
            with exp.section(
                uid=("load"),
                length=times_map['pulse_time'],
                alignment=SectionAlignment.LEFT,
            ):
                exp.play(signal="gate1",pulse=compress_level_pulse,amplitude=0.1)
                exp.play(signal="gate2",pulse=compress_level_pulse,amplitude=-0.1)
            with exp.section(
                uid=("measure"),
                length=times_map['pulse_time'],
                alignment=SectionAlignment.LEFT, 
            ): 
                exp.play(signal="gate1",pulse=compress_level_pulse,amplitude=read_amplitude[0])
                exp.play(signal="gate2",pulse=compress_level_pulse,amplitude=read_amplitude[1])
                
        if received_variable == 0:
            pulse_name = show_pulse_sheet(config['DataPaths']['data_save_path']+r'\\laboneq_output\\'+config['DataPaths']['experiment_name'], session.compiled_experiment)
            qc.config.add('pulsemap', pulse_name, value_type='object', description='laboneq pulse path', default=None)

            #shortcut to the logical signal group q0
            lsg = device_setup.logical_signal_groups["q0"].logical_signals

            #define signal map
            map_signals ={
                "gate1" : lsg["fg4_line"],
                "gate2" : lsg["fg6_line"]
            }

            noOfCaptures = NUM_REP
            
        status,timeIntervalns = pm.capture_rapid_data_block(chandle, preTriggerSamples, postTriggerSamples, int(params['timebase']), maxSamples,noOfCaptures)

        # set experiment calibration and signal map
        exp.set_signal_map(map_signals)
            
        print("Loaded exp")
        session.run(exp)
        print("Running exp finished")

        # # Check for data collection to finish using ps5000aIsReady
        status = pm.check_ready(chandle, status)

        source = ps.PS5000A_CHANNEL["PS5000A_CHANNEL_B"]
        buffersMax, buffersMin = pm.create_rapid_buffer(chandle, source, maxSamples, noOfCaptures)
        pm.getValuesRapid(chandle,maxSamples,noOfCaptures)

        # # Create time data
        time_array = pm.create_time_data(maxSamples, timeIntervalns)

        for i in range(noOfCaptures):
            datasaver.add_result((gate_instr.ch1, read_amplitude[0]),(gate_instr.ch2, read_amplitude[1]), (data_acq_mock.v1, adc2mV(buffersMax[i], chARange, maxADC)))

        dataset = datasaver.dataset
        # Stop the scope
        status = pm.stop_picoscope(chandle,status)
# Close unit Disconnect the scope 

In [ ]:
# How to get data example
dataset.get_parameter_data()

In [ ]:
interactive_psv(compiled_exp)
pm.close_picoscope(chandle,status)